Polynomials turn out to be critically important for modern cryptography. You might not have seen polynomials in a long time, so we'll review the basic concepts of polynomials from first principles. First, recall that a "function" is a rule for transforming an input from one representation to another. The concept of "function" is pretty similar in both computer science and mathematics, so let's look at some simple examples of functions in Python.

In [1]:
def f(x):
  return x + 1
print(f(1))
print(f(2))

2
3


This seems pretty simple right? How can we write `f` in mathematical notation? It's pretty simple too
$$ f(x) = x + 1 $$
Ok, let's look at some more complicated functions.

In [2]:
def g(x):
  return x**3 + x**2 + x - 1
print(g(5))

154


Mathematically, we would write `g` as a function as follows
$$g(x) = x^3 + x^2 + x - 1$$
Note that there's something simple about the function `f` and `g`. In particular, they only use a few simple operations like addition, subtraction, and multiplication. (Exponentiation is just repeated multiplication). More generally, any function that uses these simple operations is called a "polynomial." Let's take a look at a few examples of polynomials
$$h(x) = x^2$$
$$\ell(x) = (x + 7)^4 + (x-2)^2 - x$$
Alright, this makes sense. Polynomials are a simple class of function that accepts in numbers and computes operations on them. But why should we care? Well, things get interesting we we start to combine the notion of polynomials with the ideas about finite fields that we learned in the previous tutorial.

So why are polynomials interesting for computer science? The key concept is that of "arithmetization." A core idea is that any computation can be transformed into a a polynomial. Why does this make sense? Well, first let's talk about digital circuits. Any computation can be encoded as a "digital circuit." What's a digital circuit you ask? Well, you can think of it as a function built up from boolean primitives such as AND, OR, NOT and the like.

![digital_gates](https://www.oreilly.com/library/view/introduction-to-digital/9780470900550/images/ch003-f002.jpg)

These gates can be wired into one another to form a circuit. 

![digital_circuit](http://www.edwardbosworth.com/My5155_Slides/Chapter03/DigitalCircuitsAndBooleanExpressions_files/image002.jpg)

It takes a little imagining, but any algorithm can be transformed into a digital circuit. Why does this make sense? Well just reasoning from first principles we know that all algorithms we write are run on physical chips. Most chips themselves are simply large digital circuits that have the capacity to process "instructions"

![mips_circuit](https://i.imgur.com/6R3Xz.png)

In fact it's well known in computer science that you can transform back and forth between Turing Machines and Circuits as models of computation so we're on very solid ground here. But what does this have to do with polynomials? Well we explained earlier that polynomials are made up of compositions of simple operations. So are circuits. The connection comes through the idea of "arithmetic circuits"

![arithmetic_circuit](https://upload.wikimedia.org/wikipedia/commons/thumb/6/64/ArithmeticCircuit.svg/527px-ArithmeticCircuit.svg.png)

An arithmetic circuit is simply a circuit that uses addition, subtraction, and multiplication instead of boolean operators. In addition, the input values to the circuit are numbers and not booleans. (For our applications, these numbers will come from finite fields).

Let's play with the arithmetic circuit we've displayed above a bit more. For example, how can we create a function that implements this arithmetic circuit? It turns out to be relatively straightforward.

In [3]:
def f(x_1, x_2):
  return (x_1 + x_2)*(x_2+1)
f(1, 2)

9

Ok, this is relatively neat, but how is this function related to polynomials. It turns out that with a bit of simplification, we can rewrite `f` as a polynomial

$$f(x_1, x_2) = (x_1 + x_2)(x_2 + 1) = x_2^2 + x_1x_2 + x_1 + x_2$$

This is a multivariate polynomial of degree 2. So we've just seen three ways of representing the same function. First as an arithmetic circuit, then as a function, then finally as a polynomial. Congrats! You've just "arithmetized" your first computation. This notion of arithmetization is one that comes up over and over again in STARK world. You'll see it again soon enough.

Ok, we've spent most of this chapter talking about polynomials and circuits in general. How is this material related to the finite field concepts you learned about in the previous tutorial? Well to start, it will be useful to introduce some notation. How do we refer to the set of polynomials? Well first, we have to start by specifying what the coefficients are. All the polynomials we've seen till now have had integer coefficients. The set of all such polynomials is specified by

$$\mathbb{Z}[X]$$

You can read this as "Zee of X". This is the collection of all polynomials in one variable that take integers as coefficients. But what about the polynomial we just saw above that had two variables? Well we can write the set of all polynomials with integer coefficients with two variables as 

$$\mathbb{Z}[X_1, X_2]$$

You can read this as "Zee of X-one, X-two". There's one more change we have to make to allow things to get interesting. It turns that instead of using integer coefficients we can use finite field elements as coefficients. For example, the set of all polynomials with coefficients in the finite field of 7 elements is

$$\mathbb{Z}_7[X]$$

These "finite field polynomials" end up being really interesting for all sorts of cryptographic reasons. 

Before we go too deep, let's stop and get some code working. How can we define some polynomials over finite fields with the STARKs library?

In [6]:
import starks
from starks.modp import IntegersModP
from starks.polynomial import polynomials_over

mod7 = IntegersModP(7)                                                                            
polysMod7 = polynomials_over(mod7).factory                                                        

What do these lines do? Well first, we define the finite field with modulus 7
$$\mathbb{Z}_7$$
as we did in the past tutorial. We then make a factory function using `polynomials_over`. This allows us to create polynomials in
$$\mathbb{Z}_7[X]$$
Let's give it a whirl

In [8]:
x = polysMod7([0, 1])
x

0 + 1 x^1

How did this work? Well, the coefficients of a polynomial uniquely define the polynomial. So to create a polynomial, simply pass in the list of coefficients to the factory method we defined previously. You might pause here and protest that something's a bit off. 0 and 1 are integers right? How then are we defining polynomials over a finite field? Let's try and see what happens when we pass arbitrary integers in

In [9]:
polysMod7([100, 100])

2 + 2 x^1

Wait, what happened here? It turns out that `100 % 7 == 2`. So underneath the hood, the factory method is using casting to turn any integers we pass in into finite field elemements! Make sure to keep this in mind as you work with the library.

Alright, so we now know how to definie some simple polynomials. What can we do with them? Well due to operator overloading in Python, we can do all sorts of fun arithmetic operations. Let's give it a try.

In [10]:
x + x

0 + 2 x^1

In [11]:
x + 1

1 + 1 x^1

In [ ]:
x**2 + x + 2

Note that we can even treat polynomials as functions

In [15]:
g = x**2 + x + 2
g(1)

4 (mod 7)

## Exercises

1. Consider the following simple arithmetic circuit:

![three_var_circuit](https://www.researchgate.net/profile/Christos_Nasikas/publication/326657941/figure/fig4/AS:653069704523777@1532715359747/A-simple-arithmetic-circuit.png)

Convert this circuit into a function. Next, rewrite this function as a polynomial in three variables.

2. Consider the following function

$$f(x) = x^6 + x^5 + 3x^3 + 2x^2 + 1 \in \mathbb{Z}_{11}[X]$$

Define `f` as a object in python. How do you initialize the finite field? How do you create the polynomial factory? How do you define the polynomial?